# Sentence Reconstruction

The purpose of this project is to take in input a sequence of words corresponding to a random permutation of a given english sentence, and reconstruct the original sentence. 

The otuput can be either produced in a single shot, or through an iterative (autoregressive) loop generating a single token at a time.

CONSTRAINTS:
* No pretrained model can be used.
* The neural network models should have less the 20M parameters.


# Dataset

The dataset is composed by a snapshot of wikipedia. We restricted the vocabolary to the 10K most frequent words, and only took sentences making use of this vocabulary. In addition, we restricted to sequences with a length between 3 and 30 words.

(Ignore the error, if any) 

In [2]:
!pip install datasets
!pip3 install apache-beam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.48.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.1.1-py3-none-any.whl
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

In [3]:
from random import Random

# Instantiate the Random instance with random seed = 42 to ensure reproducibility
randomizer = Random(42)

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical, pad_sequences
import numpy as np 
import pickle
import gdown
import random

In [5]:
from datasets import load_dataset

dataset = load_dataset("wikipedia", "20220301.simple")

data = dataset['train'][:20000]['text']

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
#run this cell only the first time to create and save the tokenizer and the date
dump = True

tokenizer = Tokenizer(split=' ', filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', num_words=10000, oov_token='<unk>')

corpus = []

# Split of each piece of text into sentences
for elem in data:
  corpus += elem.lower().replace("\n", "").split(".")[:]

print("corpus dim: ",len(corpus))

#add a start and an end token
corpus = ['<start> '+s+' <end>' for s in corpus]

# Tokenization	
tokenizer.fit_on_texts(corpus)

if dump:
    with open('tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

original_data = [sen for sen in tokenizer.texts_to_sequences(corpus) if (len(sen) <= 32 and len(sen)>4 and not(1 in sen))]

if dump:
    with open('original.pickle', 'wb') as handle:
        pickle.dump(original_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

print ("filtered sentences: ",len(original_data))

sos = tokenizer.word_index['<start>']
eos = tokenizer.word_index['<end>']

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

corpus dim:  510023
filtered sentences:  137301


In [7]:
vocabulary_dim=(len(tokenizer.word_index)) #this dictionary maps integers to words. its size is the number of unique tokens
print(vocabulary_dim)
vocabulary_dim=10000

271573


We now create two additional datasets. 
* shuffled_data contains scrumbled sequences, and will be the input to the model. 
* target_data is the same as original data but offset by one timestep.
It is only useful if you plan to do some language modeling with a teacher forcing technique. You might decide to ignore it.


In [8]:
shuffled_data = [random.sample(s[1:-1],len(s)-2) for s in original_data]
shuffled_data = [[sos]+s+[eos] for s in shuffled_data]
target_data = [s[1:] for s in original_data] #it's original data without sos token

Let us look at some examples:

In [9]:
i = np.random.randint(len(original_data))
print("original sentence: ",original_data[i])
print("shuffled sentecen: ",shuffled_data[i])

original sentence:  [2, 251, 3587, 41, 105, 149, 75, 1985, 3]
shuffled sentecen:  [2, 3587, 251, 149, 41, 1985, 75, 105, 3]


Let us look at detokenized data:

In [10]:
i = np.random.randint(len(original_data))
print("original sentence: ",tokenizer.sequences_to_texts([original_data[i]])[0])
print("shuffled sentence: ",tokenizer.sequences_to_texts([shuffled_data[i]])[0])



original sentence:  <start> the soundtrack did better than the movie <end>
shuffled sentence:  <start> than the movie better soundtrack did the <end>


You goal is to reconstruct the original sentence out of the shuffled one.

# Additional material

Here we provide a few additional functions that could be useful to you.

As usual, you are supposed to divide your data in training and test set. Reserve at least 30% of data for testing.

You are likely to need a validation set too.

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, c_train, c_test, y_train, y_test = train_test_split(original_data, shuffled_data, target_data, test_size = 0.3, random_state = 42)


Depending from the model you plan to build, you might require padding the input sequence

In [12]:
max_sequence_len = max([len(x) for x in original_data])

x_train = pad_sequences(x_train, maxlen=max_sequence_len, padding='post')
x_test = pad_sequences(x_test, maxlen=max_sequence_len, padding='post')
c_train = pad_sequences(c_train, maxlen=max_sequence_len, padding='post')
c_test = pad_sequences(c_test, maxlen=max_sequence_len, padding='post')
y_train = pad_sequences(y_train, maxlen=max_sequence_len, padding='post')
y_test = pad_sequences(y_test, maxlen=max_sequence_len, padding='post')

In [13]:
def get_positional_vector(data, shuffled_data):
    positional_vector = np.zeros((len(data), max_sequence_len))
    for i in range(len(data)): #iterates over sentences
        for j in range(max_sequence_len): #iterates over words (tokens)
            positional_vector[i][j] = np.where(data[i] == shuffled_data[i][j])[0][0]
    return positional_vector

def get_normal_vector_from_positional(data, pos_vector):
  normal_vectors=[]
  for i in range(len(data)) #iterate over sentences
    normal_vector = np.zeros((32,))
    for j in range(32): #iterate over words (tokens)
      normal_vector[pos_vector[i][j].astype(int)]=(data[i][j])
    normal_vectors.apend(normal_vector)
  return normal_vectors


In [14]:
test_positional=get_positional_vector(x_test, c_test)
positional_vector=get_positional_vector(x_train, c_train)

In [15]:
print(positional_vector[0])
print(c_train[0])
print(x_train[0])

[0. 3. 5. 6. 7. 4. 2. 1. 8. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9.
 9. 9. 9. 9. 9. 9. 9. 9.]
[   2    8 1765    7 5114 5249   21 1534    3    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[   2 1534   21    8 5249 1765    7 5114    3    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


[   2 1534   21    8 5249 1765    7 5114    3    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


In [ ]:
from keras.utils import to_categorical
onehot_positional=to_categorical(positional_vector, num_classes=32)


In [ ]:
print("x_train size:", len(x_train))
assert(len(x_train)==len(c_train)==len(y_train))

Let us finally have a look at the distribution of data w.r.t. their lenght.

In [ ]:
import matplotlib.pyplot as plt
plt.hist([len(x)-2 for x in original_data],27)

# Metrics

Let s be the source string and p your prediction. The quality of the results will be measured according to the following metric:

1.  look for the longest substring w between s and p
2.  compute |w|/|s|

If the match is exact, the score is 1. 

When computing the score, you should NON consider the start and end tokens.



The longest common substring can be computed with the SequenceMatcher function of difflib, that allows a simple definition of our metric.

In [ ]:
from difflib import SequenceMatcher

def score(s,p):
  match = SequenceMatcher(None, s, p).find_longest_match()
  #print(match.size)
  return (match.size/len(p))

Let's do an example.

In [ ]:
original = "at first henry wanted to be friends with the king of france"
generated = "henry wanted to be friends with king of france at the first"

print("your score is ",score(original,generated))

In [ ]:
print("score taken from dataset:",score(tokenizer.sequences_to_texts([original_data[13]])[0], tokenizer.sequences_to_texts([shuffled_data[13]])[0]))

The score must be computed as an average of at least 10K random examples taken form the test set.

# What to deliver

You are supposed to deliver a single notebook, suitably commented. 
The notebook should describe a single model, although you may briefly discuss additional attempts you did.

The notebook should contain a full trace of the training. 
Weights should be made available on request.

You must also give a clear assesment of the performance of the model, computed with the metric that has been given to you.

# Good work!

In [ ]:
from tensorflow.keras import backend as K
import tensorflow as tf
def integer_accuracy(y_true, y_pred):
    y_pred_int = tf.cast(y_pred, dtype=tf.int32)
    y_true_int = tf.cast(y_true, dtype=tf.int32)

    #y_pred_int = y_pred.astype(int)
    return K.mean(K.equal(y_true_int, y_pred_int))

In [ ]:
from tensorflow.keras.layers import MultiHeadAttention, Reshape, Concatenate, LeakyReLU, Input, Embedding, LSTM, Dense, MultiHeadAttention, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import AdamW

def getLinearModel(input_dim, embedding_dim=128):
  inputs = Input(shape=(input_dim,))
  embedding = Embedding(input_dim=270000, output_dim=64)(inputs)
  lstm1, _,_ = LSTM(embedding_dim, return_sequences=True, return_state=True, recurrent_dropout=0, activation="tanh")(embedding)
  lstm2, _, _ = LSTM(embedding_dim, return_sequences=True, return_state=True, recurrent_dropout=0, activation="tanh")(lstm1)
  lstm3, _, _ = LSTM(embedding_dim, return_sequences=True,return_state=True, recurrent_dropout=0, activation="tanh")(lstm2)
  attention = MultiHeadAttention(num_heads=8, key_dim=embedding_dim)(lstm3, lstm3)
  x = Concatenate()([lstm3, attention])
  x = Dense(1024, activation=LeakyReLU(alpha=0.1))(x)
  x = Dropout(0.2)(x)
  x = Dense(512, activation=LeakyReLU(alpha=0.1))(x)
  x = Dropout(0.2)(x)
  x = Dense(128, activation=LeakyReLU(alpha=0.1))(x)
  x = Dropout(0.2)(x)
  x = Dense(128, activation=LeakyReLU(alpha=0.1))(x)
  x = Dropout(0.2)(x)
  outputs = Dense(32, activation="softmax")(x)

  model = Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
linearModel=getLinearModel((max_sequence_len))
linearModel.compile(optimizer=AdamW(0.001), loss="categorical_crossentropy", metrics=["categorical_accuracy"])
linearModel.summary()

In [ ]:
linearModel.fit(c_train, onehot_positional, epochs=20, batch_size=128, validation_split=0.1, shuffle=True)


In [ ]:
positional_vector_predictions=linearModel.predict(c_test[0:100])

In [ ]:
i = np.random.randint(100)
print(np.argmax(positional_vector_predictions[i], axis=-1))
print(test_positional[i].astype(int))

In [ ]:
print(integer_accuracy(test_positional[0:100], positional_vector_predictions))

In [ ]:
normal_vectors = []
for i in range(len(positional_vector_predictions)): #iterate over sentences
  normal_vector = []
  for j in range(len(c_train[i])): #iterate over words
    normal_vector.append(c_train[i][np.where(positional_vector_predictions[i] == j)[0][0]])
  normal_vectors.append(normal_vector)